In [1]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import pickle

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization

In [3]:
DATADIR = "E:/Django/DATA_SET/www_kaggle_com_vipoooool_new-plant-diseases-dataset/valid"
CATAGORIES = os.listdir(DATADIR)        

In [6]:
trainning_data = []
def create_trainning_data(CATAGORIES):
    IMG_SIZE = 250
    for catagory in tqdm(CATAGORIES):
        path = os.path.join(DATADIR, catagory)
        label = CATAGORIES.index(catagory)
        for img in (os.listdir(path)):
            img_arr = cv2.imread(os.path.join(path, img), cv2.COLOR_BGR2HSV)
            new_arr = cv2.resize(img_arr, (IMG_SIZE, IMG_SIZE))
            trainning_data.append([new_arr, label])        
create_trainning_data(CATAGORIES)

NameError: name 'CATAGORIES' is not defined

In [5]:
random.shuffle(trainning_data)

In [6]:
X = []
Y = []
for feature, label in trainning_data:
    X.append(feature)
    Y.append(label)

X = np.array(X).reshape(-1, 250, 250, 3)
Y = np.array(Y)
for i in tqdm(X):
    i = i/255
print("done")

100%|█████████████████████| 17572/17572 [00:20<00:00, 856.80it/s]


done


In [12]:
pickle.dump(X, open("X.pickle", "wb"))
pickle.dump(Y, open("Y.pickle", "wb"))

In [4]:
X = pickle.load(open("X.pickle", "rb"))
Y = pickle.load(open("Y.pickle", "rb")) 

In [5]:
model = Sequential([
    
    Conv2D(filters = 24, kernel_size=(11,11), activation = "relu", strides = 4 ,input_shape = (250,250,3)),
    MaxPooling2D((3,3), strides = 2),
    
    BatchNormalization(),
    
    Conv2D(filters = 64, kernel_size=(5,5), strides=1, padding="same",activation = "relu"),
    MaxPooling2D((3,3), strides=2),
    
    BatchNormalization(),
    
    Conv2D(filters = 96, kernel_size=(3,3), activation = "relu", strides=1, padding="same"),
    Conv2D(filters = 96, kernel_size=(3,3), activation = "relu", strides=1, padding="same"),
    Conv2D(filters = 66, kernel_size=(3,3), activation = "relu", strides=1, padding="same"),
    
    MaxPooling2D((3,3), strides=2),
    
    Flatten(),
    Dense(500, activation="relu"),
    Dense(500, activation="relu"),
    Dense(len(CATAGORIES), activation="softmax")
])
model.compile(optimizer='adam',
             loss = 'sparse_categorical_crossentropy',
             metrics=['accuracy'])
dd = model.fit(X,Y,epochs=70,batch_size=32,validation_split=0.1)

Epoch 1/70
495/495 [==============================] - 107s 145ms/step - loss: 2.7132 - accuracy: 0.2515 - val_loss: 1.7378 - val_accuracy: 0.4881
Epoch 2/70
495/495 [==============================] - 14s 29ms/step - loss: 1.0991 - accuracy: 0.6562 - val_loss: 1.2742 - val_accuracy: 0.6149
Epoch 3/70
495/495 [==============================] - 14s 29ms/step - loss: 0.7343 - accuracy: 0.7578 - val_loss: 0.9800 - val_accuracy: 0.7150
Epoch 4/70
495/495 [==============================] - 14s 29ms/step - loss: 0.5462 - accuracy: 0.8200 - val_loss: 1.0334 - val_accuracy: 0.7042
Epoch 5/70
495/495 [==============================] - 14s 29ms/step - loss: 0.4096 - accuracy: 0.8710 - val_loss: 1.2208 - val_accuracy: 0.6940
Epoch 6/70
495/495 [==============================] - 14s 29ms/step - loss: 0.3501 - accuracy: 0.8904 - val_loss: 1.3530 - val_accuracy: 0.6581
Epoch 7/70
495/495 [==============================] - 14s 29ms/step - loss: 0.3008 - accuracy: 0.9046 - val_loss: 0.6167 - val_accurac

In [5]:
model.save('E:/Django/DATA_SET/trainned_desease.model')

NameError: name 'model' is not defined

In [4]:
model = keras.models.load_model('E:/Django/DATA_SET/trainned_desease.model')

In [8]:
test_data = []
DATA = "E:/Django/DATA_SET/www_kaggle_com_vipoooool_new-plant-diseases-dataset/test"
for img in (os.listdir(DATA)):
    img_arr = cv2.imread(os.path.join(DATA, img), cv2.COLOR_BGR2HSV)
    new_arr = cv2.resize(img_arr, (250, 250))
    test_data.append([new_arr, img]) 
    
X_test = []
Y_test = []
for feature, label in test_data:
    X_test.append(feature)
    Y_test.append(label)
Y_test = np.array(Y_test)
X_test = np.array(X_test).reshape(-1, 250, 250, 3)
for i in tqdm(X_test):
    i = i/255

100%|█████████████████████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 934.17it/s]


In [9]:
predicted = [np.argmax(i) for i in model.predict(X_test)]
print("CNN output ---------- Original output" )
for i in range(len(Y_test)):
    print(CATAGORIES[predicted[i]],"----------" ,Y_test[i])

CNN output ---------- Original output
Apple___Cedar_apple_rust ---------- AppleCedarRust1.JPG
Apple___Cedar_apple_rust ---------- AppleCedarRust2.JPG
Apple___Cedar_apple_rust ---------- AppleCedarRust3.JPG
Apple___Cedar_apple_rust ---------- AppleCedarRust4.JPG
Apple___Apple_scab ---------- AppleScab1.JPG
Apple___Apple_scab ---------- AppleScab2.JPG
Squash___Powdery_mildew ---------- AppleScab3.JPG
Corn_(maize)___Common_rust_ ---------- CornCommonRust1.JPG
Corn_(maize)___Common_rust_ ---------- CornCommonRust2.JPG
Corn_(maize)___Common_rust_ ---------- CornCommonRust3.JPG
Potato___Early_blight ---------- PotatoEarlyBlight1.JPG
Potato___Early_blight ---------- PotatoEarlyBlight2.JPG
Potato___Early_blight ---------- PotatoEarlyBlight3.JPG
Potato___Early_blight ---------- PotatoEarlyBlight4.JPG
Potato___Early_blight ---------- PotatoEarlyBlight5.JPG
Potato___healthy ---------- PotatoHealthy1.JPG
Potato___healthy ---------- PotatoHealthy2.JPG
Tomato___Early_blight ---------- TomatoEarlyBli

In [18]:
#testing
test_data = []
DATA = "E:/Django/DATA_SET/www_kaggle_com_vipoooool_new-plant-diseases-dataset/test/TomatoEarlyBlight1.jpg"
img_arr = cv2.imread(DATA, cv2.COLOR_BGR2HSV)
new_arr = cv2.resize(img_arr, (250, 250))
new_arr = np.array(new_arr).reshape(-1, 250, 250, 3)
predicted = np.argmax(model.predict(new_arr))

In [19]:
CATAGORIES[predicted]

'Potato___healthy'

In [11]:
predicted

8

In [4]:
model = tf.saved_model.load('E:/Django/DATA_SET/trainned_desease.model')

In [5]:
model.

AttributeError: '_UserObject' object has no attribute 'predict'

In [7]:
Y.shape

(17572,)